Teknik Haberlerin Alınması

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://techcrunch.com/latest/page/{}/"
headers = {"User-Agent": "Mozilla/5.0"}

news_data = [] #dictionary içerir, haberin bilgilerini tutar

# İlk 5 sayfayı gez (her sayfada 20 haber var)
for page_num in range(1, 6):

    print(f"Sayfa {page_num} işleniyor...")
    page_url = base_url.format(page_num)
    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    #Linklerin bulunduğu h3 etiketlerini bul
    h3_tags = soup.find_all('h3', class_='loop-card__title')

    for h3 in h3_tags[:20]: # [:20] yazmadığımızda her sayfadan 27 haber çekiyor (yedeklenmiş eski içerikleri alıyor)
        a_tag = h3.find('a')
        if a_tag and a_tag.has_attr('href'):
            news_url = a_tag['href']
            print("Haber çekiliyor:", news_url)

            try:
                #1-HABER SAYFASINI ALMA
                news_response = requests.get(news_url, headers=headers)
                news_soup = BeautifulSoup(news_response.text, "html.parser")


                #2-HABER İÇERİĞİNİ ALMA
                content_div = news_soup.find('div', class_='entry-content wp-block-post-content is-layout-constrained wp-block-post-content-is-layout-constrained')
                if content_div:
                    p_tags = content_div.find_all('p', class_='wp-block-paragraph')

                    if p_tags:
                        # Paragrafları Birleştirme
                        paragraphs = []
                        for p in p_tags:
                            text = p.get_text(strip=True)
                            if text:
                                paragraphs.append(text)

                        # Paragraf arasında boş satır bırak
                        content = '\n\n'.join(paragraphs)
                    else:
                        content = ""  # p_tags boşsa, içerik olmamalı
                else:
                    content = ""  # content_div bulunamadıysa, içerik olmamalı

                #3-VERİYİ LİSTEYE EKLE
                if content:
                    news_data.append({
                        "Category": "Technical",
                        "Content": content
                    })

                time.sleep(1)  # her istek arasında 1 saniye bekler

            except Exception as e:
                print(f"Haber alınamadı: {news_url}\nHata: {e}")

# Verileri CSV dosyasına yaz
df = pd.DataFrame(news_data) #news_data listesi bir tabloya dönüştürülür.
df.to_csv("techCrunch_news.csv", index=False, encoding='utf-8-sig')

print("✅ 100 haber başarıyla 'techcrunch_haberler.csv' dosyasına kaydedildi.")

#NOT - Neden [:20] ifadesi kullandık?
#Sayfa altında yedeklenmiş eski içerikler
#Önbelleğe alınmış kartlar
#Ya da display:none ile gizlenmiş linkler hala sayfa kodunun içinde bulunuyor olabilir.

Sayfa 1 işleniyor...
Haber çekiliyor: https://techcrunch.com/2025/05/30/elon-musk-is-lobbying-lawmakers-on-driverless-vehicle-rules/
Haber çekiliyor: https://techcrunch.com/2025/05/30/yep-x-was-down-again/
Haber çekiliyor: https://techcrunch.com/2025/05/30/techcrunch-mobility-a-ride-sharing-pioneer-comes-for-uber-tesla-loses-more-ground-and-dog-like-delivery-robots-land-in-texas/
Haber çekiliyor: https://techcrunch.com/2025/05/30/doge-left-united-states-institute-of-peace-office-with-water-damage-rats-and-roaches/
Haber çekiliyor: https://techcrunch.com/2025/05/30/its-not-your-imagination-ai-is-speeding-up-the-pace-of-change/
Haber çekiliyor: https://techcrunch.com/2025/05/30/trump-administration-to-claw-back-3-7b-in-clean-energy-and-manufacturing-awards/
Haber çekiliyor: https://techcrunch.com/2025/05/30/automattic-says-it-will-start-contributing-to-wordpress-again-after-pause/
Haber çekiliyor: https://techcrunch.com/2025/05/30/startups-weekly-amd-acquisition-and-other-moves-to-scale-

Sağlık Haberlerinin Alınması


In [ ]:
base_url = "https://www.news-medical.net/medical/news?page={}"
headers = {"User-Agent": "Mozilla/5.0"}

news_data = [] #dictionary içerir, haberin bilgilerini tutar

# İlk 5 sayfayı geziyoruz (her sayfada 20 haber var)
for page_num in range(1, 6):

    print(f"Sayfa {page_num} işleniyor...")
    page_url = base_url.format(page_num)
    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Linklerin Bulunduğu h3 etiketlerini bul
    div = soup.find('div', class_="posts publishables-list-wrap first-item-larger")
    h3_tags = div.find_all('h3')

    for h3 in h3_tags[:20]:
        a_tag = h3.find('a')
        if a_tag and a_tag.has_attr('href'):
            news_url = "https://www.news-medical.net" + a_tag['href']
            print("Haber çekiliyor:", news_url)


            try:
                #1-HABER SAYFASINI ALMA
                news_response = requests.get(news_url, headers=headers)
                news_soup = BeautifulSoup(news_response.text, "html.parser")


                #2-HABER İÇERİĞİNİ ALMA
                content_div = news_soup.find('div', class_='page-content clearfix')
                if content_div:

                  # "sources" id'li div'i hariç tutarak sadece p ve blockquote dışı etiketleri al
                  for source_div in content_div.find_all('div', id='sources'):
                     source_div.decompose()  # Bu div'i ve içeriğini kaldır

                  # blockquote etiketlerini hariç tutmak için
                  for blockquote in content_div.find_all('blockquote'):
                     blockquote.decompose()  # blockquote etiketini kaldır

                  #p etiketlerini al
                  p_tags = content_div.find_all('p')

                  if p_tags:
                        # Paragrafları Birleştirme
                        paragraphs = []
                        for p in p_tags:
                            text = p.get_text(strip=True)
                            if text:
                                paragraphs.append(text)

                        # Paragraf arasında boş satır bırak
                        content = '\n\n'.join(paragraphs)
                  else:
                        content = ""  # p_tags boşsa, içerik olmamalı
                else:
                    content = ""  # content_div bulunamadıysa, içerik olmamalı

                #3-VERİYİ LİSTEYE EKLE
                if content:
                    news_data.append({
                        "Category": "Medicine",
                        "Content": content
                    })

                time.sleep(1)  # her istek arasında 1 saniye bekler

            except Exception as e:
                print(f"Haber alınamadı: {news_url}\nHata: {e}")

# Verileri CSV dosyasına yaz
df = pd.DataFrame(news_data) #news_data listesi bir tabloya dönüştürülür.
df.to_csv("newsMedical_news.csv", index=False, encoding='utf-8-sig')

print("✅ 100 haber başarıyla 'newsMedical_haberler.csv' dosyasına kaydedildi.")


Sayfa 1 işleniyor...
Haber çekiliyor: https://www.news-medical.net/news/20250531/Diagnosticsai-launches-fully-transparent-machine-learning-platform-for-real-time-PCR-diagnostics.aspx
Haber çekiliyor: https://www.news-medical.net/news/20250531/Gene-expression-study-reveals-clues-to-asbestos-linked-mesothelioma.aspx
Haber çekiliyor: https://www.news-medical.net/news/20250531/New-wearable-device-monitors-brain-waste-clearance-during-sleep.aspx
Haber çekiliyor: https://www.news-medical.net/news/20250531/Light-powered-phagocytic-macrophage-microrobot-combines-immune-functions-with-robotic-controllability.aspx
Haber çekiliyor: https://www.news-medical.net/news/20250531/Microfluidic-contact-lenses-emerge-as-next-generation-tools-for-eye-care.aspx
Haber çekiliyor: https://www.news-medical.net/news/20250531/New-microfluidic-device-brings-affordable-kidney-testing-to-the-point-of-care.aspx
Haber çekiliyor: https://www.news-medical.net/news/20250531/New-studies-offer-hope-for-easing-primary-care-

CSV Dosyalarının Birleştirilmesi



In [ ]:
# 1. Teknik ve medikal haber CSV'lerini oku
tech_news_df = pd.read_csv('techCrunch_news.csv', encoding='utf-8-sig')
medicine_news_df = pd.read_csv('newsMedical_news.csv', encoding='utf-8-sig')

# 2. Bu iki veri kümesini birleştir
combined_df = pd.concat([tech_news_df, medicine_news_df], ignore_index=True)

# 3. combined_news.csv olarak kaydet (isteğe bağlı ara kayıt)
combined_df.to_csv('combined_news.csv', index=False, encoding='utf-8-sig')

# 4. Ambiguous haber veri kümesini oku
ambiguous_df = pd.read_csv('ambiguous_news.csv', encoding='utf-8-sig')

# 5. combined_news + ambiguous_news birleşimi
final_df = pd.concat([combined_df, ambiguous_df], ignore_index=True)

# 6. Son birleşik veri kümesini kaydet
final_df.to_csv('final_news.csv', index=False, encoding='utf-8-sig')

print("✅ Tüm haberler başarıyla birleştirildi ve 'final_news.csv' olarak kaydedildi.")



✅ Tüm haberler başarıyla birleştirildi ve 'final_news.csv' olarak kaydedildi.
